In [1]:
import pandas as pd
import plotly.express as px
import seaborn as sns
import datetime
import numpy as np

In [2]:
# Serie de tiempo
ventas = pd.read_csv('femsa_venta_historica.csv')

femsa = pd.read_csv('femsa_oro_plata.csv')
# Renombrar columnas
femsa = femsa.rename(columns = {'Coordenada X': 'Longitud'})
femsa = femsa.rename(columns = {'Coordenada Y': 'Latitud'})
# Excluir oro, dejar solo plata y plata +
femsa = femsa[femsa.Segmento != 'Oro']
# Dejar solo usuarios naturales
femsa['tipo_usuario'] = ['empresa' if str(x)[:2]=='90' else 'natural' for x in list(femsa['Cédula'])]
femsa = femsa[femsa['tipo_usuario']=='natural']

##### FILTRO 3
# Agregar segmento y dejar solo plata y plata + naturales
ventas = pd.merge(ventas, femsa[['Cliente', 'Segmento',"Longitud", "Latitud","Jefatura","Gerencia","Locacion","tipo_usuario","Canal"]], on ='Cliente', how = 'left').dropna(subset=['Segmento'])

##### FILTRO 1
# Dejar solo los que tengan muestra antes de los 6 meses
antes_6_meses = ventas[ventas['Nombre Mes'] < 7].Cliente.unique().tolist()
ventas = ventas[ventas.Cliente.isin(antes_6_meses)]

##### FILTRO 2
# Dejar solo los que tengan ventas en los últimos 3 meses
ultimos_3_meses = ventas[ventas['Nombre Mes'] > 9].Cliente.unique().tolist()
ventas = ventas[ventas.Cliente.isin(ultimos_3_meses)]

# Sacar el ingreso bruto por mes de cada cliente

# Corte transversal

# Importar cupos actuales de los clientes
cupos_clientes = pd.read_csv('cupos_clientes.csv')
cupos_clientes = cupos_clientes.rename(columns={'CLIENTE':'Cliente'})
# Agregar el cupo de cada cliente
ventas = pd.merge(ventas, cupos_clientes, on ='Cliente', how = 'left')

#### FILTRO 4
# informal suspendido
ventas = ventas[~ventas['CLASE DE RIESGO'].isin(['INFORMAL SUSPENDIDO'])]

# Importar carteras corrientes y vencidas
cartera = pd.read_csv('cartera_08_03_22.csv')
# Importar los reportados en credikof
reportados_credikof = pd.read_csv('reportados_credikof.csv')

# Sacar lista de clientes con cartera vencida y reportados
clientes_vencida = cartera[cartera['Estado_Vcdo'] == 'Vencida'].drop_duplicates('Cliente').Cliente.tolist()
clientes_reportados = reportados_credikof.drop_duplicates('CLIENTE').CLIENTE.tolist()

##### FILTRO 5
# Eliminar clientes con cartera vencida y reportados
print('clientes plata/plata + con cartera vencida: ', len(ventas[ventas.Cliente.isin(clientes_vencida)].drop_duplicates('Cliente')))
ventas = ventas[~ventas.Cliente.isin(clientes_vencida)]
print('clientes plata/plata +  reportados: ', len(ventas[ventas.Cliente.isin(clientes_reportados)].drop_duplicates('Cliente')))
ventas = ventas[~ventas.Cliente.isin(clientes_reportados)]


clientes plata/plata + con cartera vencida:  55
clientes plata/plata +  reportados:  51


In [3]:
## haciendo los cupos por medio de las dos metodologias definidas
agrupado=ventas.groupby(by=["Cliente","Latitud","Longitud","Segmento","Locacion","Nombre Mes"],as_index=False)["Ingreso Bruto 1"].sum()
agrupado["ventas_promedio_3"]=agrupado.groupby(by=["Cliente","Latitud","Longitud","Segmento","Locacion"])["Ingreso Bruto 1"].rolling(3).mean().tolist()
agrupado["ventas_maximas"]=agrupado.groupby(by=["Cliente","Latitud","Longitud","Segmento","Locacion"])["Ingreso Bruto 1"].cummax().tolist()

agrupado.dropna(inplace=True)

In [4]:

agrupado["cupo_mean"]=agrupado["ventas_promedio_3"]*(1.3)
agrupado["cupo_max"]=agrupado["ventas_maximas"]*(1.3)
agrupado["Nombre Mes"]=agrupado["Nombre Mes"]+1

In [5]:
ventas_cupo=pd.merge(ventas,agrupado[["Cliente","Nombre Mes","cupo_mean","cupo_max"]],on=["Cliente","Nombre Mes"],how="left").dropna()
ventas_cupo.dropna(subset=["Nombre Mes"],inplace=True)

In [6]:
ventas_cupo['cupo_mean_truncado'] = round(ventas_cupo["cupo_mean"]/100000,0)*100000
ventas_cupo["cupo_max_truncado"] = round(ventas_cupo["cupo_max"]/100000,0)*100000

#### Sacar columna de volatilidad y filtrar por eso.
#### MIrar la localidad y el tipo de tienda.
#### Numero de pedidos que tiene en ese historico.
#### graficar la suma y el promedio del cupo de los 200 seleccionado.
#### Mirar tambien, maximo, minimo y percentiles.

In [7]:
ventas_cupo

,Cliente,Nombre,Nombre del cliente,Año,Nombre Mes,Día natural,Ingreso Bruto 1,Segmento,Longitud,Latitud,...,Locacion,tipo_usuario,Canal,CUPO,CLASE DE RIESGO,PLAZO DIAS,cupo_mean,cupo_max,cupo_mean_truncado,cupo_max_truncado
20,1200288807,TIENDA LUCY,VACA RAMIREZ JAIME ALFREDO,2021,4,2021-04-03,107353,Plata +,-74.1464,4.5966,...,Bogota Sur,natural,TIENDA DE BARRIO,0,CONTADO,0,1235339.3,1540479.2,1200000.0,1500000.0
21,1200288807,TIENDA LUCY,VACA RAMIREZ JAIME ALFREDO,2021,4,2021-04-07,100083,Plata +,-74.1464,4.5966,...,Bogota Sur,natural,TIENDA DE BARRIO,0,CONTADO,0,1235339.3,1540479.2,1200000.0,1500000.0
22,1200288807,TIENDA LUCY,VACA RAMIREZ JAIME ALFREDO,2021,4,2021-04-10,116672,Plata +,-74.1464,4.5966,...,Bogota Sur,natural,TIENDA DE BARRIO,0,CONTADO,0,1235339.3,1540479.2,1200000.0,1500000.0
23,1200288807,TIENDA LUCY,VACA RAMIREZ JAIME ALFREDO,2021,4,2021-04-14,87479,Plata +,-74.1464,4.5966,...,Bogota Sur,natural,TIENDA DE BARRIO,0,CONTADO,0,1235339.3,1540479.2,1200000.0,1500000.0
24,1200288807,TIENDA LUCY,VACA RAMIREZ JAIME ALFREDO,2021,4,2021-04-17,100820,Plata +,-74.1464,4.5966,...,Bogota Sur,natural,TIENDA DE BARRIO,0,CONTADO,0,1235339.3,1540479.2,1200000.0,1500000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323981,1224316676,FRANCHOS PIZZA LOS ANGELES,ARCHILA GONZALEZ FRANCISCO JAVIER,2021,12,2021-12-15,93445,Plata +,-74.1192,4.6987,...,Bogota Norte,natural,COMIDAS RAPIDAS,0,CONTADO,0,1123643.3,1402164.4,1100000.0,1400000.0
323982,1224316676,FRANCHOS PIZZA LOS ANGELES,ARCHILA GONZALEZ FRANCISCO JAVIER,2021,12,2021-12-18,219918,Plata +,-74.1192,4.6987,...,Bogota Norte,natural,COMIDAS RAPIDAS,0,CONTADO,0,1123643.3,1402164.4,1100000.0,1400000.0
323983,1224316676,FRANCHOS PIZZA LOS ANGELES,ARCHILA GONZALEZ FRANCISCO JAVIER,2021,12,2021-12-22,152386,Plata +,-74.1192,4.6987,...,Bogota Norte,natural,COMIDAS RAPIDAS,0,CONTADO,0,1123643.3,1402164.4,1100000.0,1400000.0
323984,1224316676,FRANCHOS PIZZA LOS ANGELES,ARCHILA GONZALEZ FRANCISCO JAVIER,2021,12,2021-12-24,51907,Plata +,-74.1192,4.6987,...,Bogota Norte,natural,COMIDAS RAPIDAS,0,CONTADO,0,1123643.3,1402164.4,1100000.0,1400000.0


In [8]:
print("cantidad de casos en las que el cupo no seria suficiente con la media:",sum(ventas_cupo["Ingreso Bruto 1"]>ventas_cupo["cupo_mean"]))
print("cantidad de casos en las que el cupo no seria suficiente con el maximo:",sum(ventas_cupo["Ingreso Bruto 1"]>ventas_cupo["cupo_max"]))
print("cantidad de casos en las que el cupo no seria suficiente con la media truncada:",sum(ventas_cupo["Ingreso Bruto 1"]>ventas_cupo["cupo_mean_truncado"]))
print("cantidad de casos en las que el cupo no seria suficiente con el maximo truncado:",sum(ventas_cupo["Ingreso Bruto 1"]>ventas_cupo["cupo_max_truncado"]))

cantidad de casos en las que el cupo no seria suficiente con la media: 364
cantidad de casos en las que el cupo no seria suficiente con el maximo: 97
cantidad de casos en las que el cupo no seria suficiente con la media truncada: 386
cantidad de casos en las que el cupo no seria suficiente con el maximo truncado: 107


In [9]:
print("Monto que no se podria cumplir con el cupo de la media:",sum((ventas_cupo["Ingreso Bruto 1"]-ventas_cupo["cupo_mean"])[ventas_cupo["Ingreso Bruto 1"]>ventas_cupo["cupo_mean"]]))
print("Monto que no se podria cubrir con el cupo del maximo:",sum((ventas_cupo["Ingreso Bruto 1"]-ventas_cupo["cupo_max"])[ventas_cupo["Ingreso Bruto 1"]>ventas_cupo["cupo_max"]]))
print("Monto que no se podria cumplir con el cupo de la media truncado:",sum((ventas_cupo["Ingreso Bruto 1"]-ventas_cupo["cupo_mean_truncado"])[ventas_cupo["Ingreso Bruto 1"]>ventas_cupo["cupo_mean_truncado"]]))
print("Monto que no se podria cubrir con el cupo del maximo truncado:",sum((ventas_cupo["Ingreso Bruto 1"]-ventas_cupo["cupo_max_truncado"])[ventas_cupo["Ingreso Bruto 1"]>ventas_cupo["cupo_max_truncado"]]))


Monto que no se podria cumplir con el cupo de la media: 82444845.99999996
Monto que no se podria cubrir con el cupo del maximo: 29705337.299999993
Monto que no se podria cumplir con el cupo de la media truncado: 84116277.0
Monto que no se podria cubrir con el cupo del maximo truncado: 30096221.0


In [10]:
## Sacando que tan cerca ha estado de alcanzar el cupo maximo que se le asigno en cada momento del tiempo.
Sobre_cupo=pd.DataFrame(ventas_cupo.groupby(by="Cliente").apply(lambda x: (x["cupo_mean_truncado"]-x["Ingreso Bruto 1"]).mean())).reset_index()
Sobre_cupo.columns=["Cliente","indicador"]
## Sacar la desviación estandar pero con respecto a la media movil de 3 meses de las ventas del cliente.
Sobre_cupo["Std_ventas"]=agrupado.groupby(by="Cliente").apply(lambda x: ((x["Ingreso Bruto 1"]-x["ventas_promedio_3"])**2).mean())

In [12]:
agrupado.groupby(by="Cliente").apply(lambda x: ((x["Ingreso Bruto 1"]-x["ventas_promedio_3"])**2).mean())

Cliente
1200003341    1.490710e+10
1200003608    4.503826e+09
1200003613    2.411432e+09
1200003619    2.761615e+10
1200003814    8.609123e+08
                  ...     
1224315451    1.381106e+10
1224315642    2.975437e+10
1224316636    8.791955e+10
1224316676    3.378718e+10
1224316705    6.453764e+09
Length: 4589, dtype: float64

In [13]:
len(ventas_cupo.Cliente[ventas_cupo["Ingreso Bruto 1"]>ventas_cupo["cupo_max"]].unique())/len(ventas_cupo.Cliente.unique())

0.017432991937241227

In [14]:
## COn la media del ingreso y los hints
ventas_cupo.groupby(by=["Segmento"]).apply(lambda x: sum(x["Ingreso Bruto 1"]>x["cupo_mean"])/len(x))

Segmento
Plata       0.002117
Plata +     0.001032
dtype: float64

In [15]:
## con la media del ingreso y el saldo
ventas_cupo.groupby(by=["Segmento"]).apply(lambda x: sum((x["Ingreso Bruto 1"]-x["cupo_mean"])[x["Ingreso Bruto 1"]>x["cupo_mean"]])/sum(x["Ingreso Bruto 1"]))

Segmento
Plata       0.005923
Plata +     0.002943
dtype: float64

In [16]:
### COn el maximo del ingreso y los hints
ventas_cupo.groupby(by=["Segmento"]).apply(lambda x: sum(x["Ingreso Bruto 1"]>x["cupo_max"])/len(x))

Segmento
Plata       0.000527
Plata +     0.000303
dtype: float64

In [17]:
## Con el maximo del ingreso y el saldo
ventas_cupo.groupby(by=["Segmento"]).apply(lambda x: sum((x["Ingreso Bruto 1"]-x["cupo_max"])[x["Ingreso Bruto 1"]>x["cupo_max"]])/sum(x["Ingreso Bruto 1"]))

Segmento
Plata       0.002212
Plata +     0.001017
dtype: float64

In [18]:
agrupado_1=agrupado[agrupado.Cliente.isin(ventas_cupo.Cliente)].groupby("Cliente").last().reset_index()


In [19]:
ventas_cupo.cupo_mean.mean()

714515.4154183193

In [20]:
agrupado_1["Ingreso Bruto 1"].mean()

589230.4842013511

In [21]:
ventas_cupo

,Cliente,Nombre,Nombre del cliente,Año,Nombre Mes,Día natural,Ingreso Bruto 1,Segmento,Longitud,Latitud,...,Locacion,tipo_usuario,Canal,CUPO,CLASE DE RIESGO,PLAZO DIAS,cupo_mean,cupo_max,cupo_mean_truncado,cupo_max_truncado
20,1200288807,TIENDA LUCY,VACA RAMIREZ JAIME ALFREDO,2021,4,2021-04-03,107353,Plata +,-74.1464,4.5966,...,Bogota Sur,natural,TIENDA DE BARRIO,0,CONTADO,0,1235339.3,1540479.2,1200000.0,1500000.0
21,1200288807,TIENDA LUCY,VACA RAMIREZ JAIME ALFREDO,2021,4,2021-04-07,100083,Plata +,-74.1464,4.5966,...,Bogota Sur,natural,TIENDA DE BARRIO,0,CONTADO,0,1235339.3,1540479.2,1200000.0,1500000.0
22,1200288807,TIENDA LUCY,VACA RAMIREZ JAIME ALFREDO,2021,4,2021-04-10,116672,Plata +,-74.1464,4.5966,...,Bogota Sur,natural,TIENDA DE BARRIO,0,CONTADO,0,1235339.3,1540479.2,1200000.0,1500000.0
23,1200288807,TIENDA LUCY,VACA RAMIREZ JAIME ALFREDO,2021,4,2021-04-14,87479,Plata +,-74.1464,4.5966,...,Bogota Sur,natural,TIENDA DE BARRIO,0,CONTADO,0,1235339.3,1540479.2,1200000.0,1500000.0
24,1200288807,TIENDA LUCY,VACA RAMIREZ JAIME ALFREDO,2021,4,2021-04-17,100820,Plata +,-74.1464,4.5966,...,Bogota Sur,natural,TIENDA DE BARRIO,0,CONTADO,0,1235339.3,1540479.2,1200000.0,1500000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323981,1224316676,FRANCHOS PIZZA LOS ANGELES,ARCHILA GONZALEZ FRANCISCO JAVIER,2021,12,2021-12-15,93445,Plata +,-74.1192,4.6987,...,Bogota Norte,natural,COMIDAS RAPIDAS,0,CONTADO,0,1123643.3,1402164.4,1100000.0,1400000.0
323982,1224316676,FRANCHOS PIZZA LOS ANGELES,ARCHILA GONZALEZ FRANCISCO JAVIER,2021,12,2021-12-18,219918,Plata +,-74.1192,4.6987,...,Bogota Norte,natural,COMIDAS RAPIDAS,0,CONTADO,0,1123643.3,1402164.4,1100000.0,1400000.0
323983,1224316676,FRANCHOS PIZZA LOS ANGELES,ARCHILA GONZALEZ FRANCISCO JAVIER,2021,12,2021-12-22,152386,Plata +,-74.1192,4.6987,...,Bogota Norte,natural,COMIDAS RAPIDAS,0,CONTADO,0,1123643.3,1402164.4,1100000.0,1400000.0
323984,1224316676,FRANCHOS PIZZA LOS ANGELES,ARCHILA GONZALEZ FRANCISCO JAVIER,2021,12,2021-12-24,51907,Plata +,-74.1192,4.6987,...,Bogota Norte,natural,COMIDAS RAPIDAS,0,CONTADO,0,1123643.3,1402164.4,1100000.0,1400000.0


In [22]:
100000*3800/686039

553.9043698681853

In [23]:
## Con la latitud y longitud hacer geolocalización inversa para sacar el barrio y eso de manera robusta y hacer un DBSCAN por densidad
## luego mirar si hay algun tipo de tienda que predomina

In [24]:
# Serie de tiempo
ventas = pd.read_csv('femsa_venta_historica.csv')
print(len(ventas.Cliente.unique()))
femsa = pd.read_csv('femsa_oro_plata.csv')
# Renombrar columnas
femsa = femsa.rename(columns = {'Coordenada X': 'Longitud'})
femsa = femsa.rename(columns = {'Coordenada Y': 'Latitud'})
# Excluir oro, dejar solo plata y plata +
femsa = femsa[femsa.Segmento != 'Oro']
# Dejar solo usuarios naturales
femsa['tipo_usuario'] = ['empresa' if str(x)[:2]=='90' else 'natural' for x in list(femsa['Cédula'])]
femsa = femsa[femsa['tipo_usuario']=='natural']

##### FILTRO 3
# Agregar segmento y dejar solo plata y plata + naturales
ventas = pd.merge(ventas, femsa[['Cliente', 'Segmento']], on ='Cliente', how = 'left').dropna(subset=['Segmento'])
print(len(ventas.Cliente.unique()))
##### FILTRO 1
# Dejar solo los que tengan muestra antes de los 6 meses
antes_6_meses = ventas[ventas['Nombre Mes'] < 7].Cliente.unique().tolist()
ventas = ventas[ventas.Cliente.isin(antes_6_meses)]
print(len(ventas.Cliente.unique()))
##### FILTRO 2
# Dejar solo los que tengan ventas en los últimos 3 meses
ultimos_3_meses = ventas[ventas['Nombre Mes'] > 9].Cliente.unique().tolist()
ventas = ventas[ventas.Cliente.isin(ultimos_3_meses)]
print(len(ventas.Cliente.unique()))
# Sacar el ingreso bruto por mes de cada cliente

# Corte transversal


print(len(ventas.Cliente.unique()))
# Importar cupos actuales de los clientes
cupos_clientes = pd.read_csv('cupos_clientes.csv')
cupos_clientes = cupos_clientes.rename(columns={'CLIENTE':'Cliente'})
# Agregar el cupo de cada cliente
ventas = pd.merge(ventas, cupos_clientes, on ='Cliente', how = 'left')

#### FILTRO 4
# Quitar contado e informal suspendido
ventas = ventas[~ventas['CLASE DE RIESGO'].isin(['INFORMAL SUSPENDIDO'])]
print(len(ventas.Cliente.unique()))
# Importar carteras corrientes y vencidas
cartera = pd.read_csv('cartera_08_03_22.csv')
# Importar los reportados en credikof
reportados_credikof = pd.read_csv('reportados_credikof.csv')

# Sacar lista de clientes con cartera vencida y reportados
clientes_vencida = cartera[cartera['Estado_Vcdo'] == 'Vencida'].drop_duplicates('Cliente').Cliente.tolist()
clientes_reportados = reportados_credikof.drop_duplicates('CLIENTE').CLIENTE.tolist()
print(len(ventas.Cliente.unique()))
##### FILTRO 5
# Eliminar clientes con cartera vencida y reportados
print('clientes plata/plata + con cartera vencida: ', len(ventas[ventas.Cliente.isin(clientes_vencida)].drop_duplicates('Cliente')))
ventas = ventas[~ventas.Cliente.isin(clientes_vencida)]
print('clientes plata/plata +  reportados: ', len(ventas[ventas.Cliente.isin(clientes_reportados)].drop_duplicates('Cliente')))
ventas = ventas[~ventas.Cliente.isin(clientes_reportados)]
print(len(ventas.Cliente.unique()))

21455
6446
5838
5837
5837
4712
4712
clientes plata/plata + con cartera vencida:  55
clientes plata/plata +  reportados:  51
4606


In [25]:
# ventas = pd.read_csv('femsa_venta_historica.csv')
femsa = pd.read_csv('femsa_oro_plata.csv')
# Renombrar columnas
femsa = femsa.rename(columns = {'Coordenada X': 'Longitud'})
femsa = femsa.rename(columns = {'Coordenada Y': 'Latitud'})
# Excluir oro, dejar solo plata y plata +
femsa = femsa[femsa.Segmento != 'Oro']
# Dejar solo usuarios naturales
femsa['tipo_usuario'] = ['empresa' if str(x)[:2]=='90' else 'natural' for x in list(femsa['Cédula'])]
femsa = femsa[femsa['tipo_usuario']=='natural']

##### FILTRO 3
# Agregar segmento y dejar solo plata y plata + naturales
ventas= pd.merge(ventas, femsa[['Cliente',"Longitud", "Latitud","Jefatura","Gerencia","Locacion","tipo_usuario"]], on ='Cliente', how = 'left').dropna(subset=['Segmento'])

In [26]:
grupos="Locacion"
# max_depth_def=3
agrupado=ventas.groupby(by=["Cliente","Latitud","Longitud","Segmento",grupos,"Nombre Mes"],as_index=False)["Ingreso Bruto 1"].sum()
agrupado["ventas_promedio_3"]=agrupado.groupby(by=[grupos,"Cliente","Latitud","Longitud","Segmento"])["Ingreso Bruto 1"].rolling(3).mean().tolist()
agrupado["ventas_maximas"]=agrupado.groupby(by=[grupos,"Cliente","Latitud","Longitud","Segmento"])["Ingreso Bruto 1"].cummax().tolist()

agrupado.dropna(inplace=True)

In [27]:
agrupado["cupo_mean"]=agrupado["ventas_promedio_3"]*(1.5)
agrupado["cupo_max"]=agrupado["ventas_maximas"]*(1.5)
agrupado["Nombre Mes"]=agrupado["Nombre Mes"]+1

In [28]:
ventas_cupo=pd.merge(ventas,agrupado[["Cliente","Nombre Mes","cupo_mean","cupo_max"]],on=["Cliente","Nombre Mes"],how="left").dropna()
ventas_cupo

,Cliente,Nombre,Nombre del cliente,Año,Nombre Mes,Día natural,Ingreso Bruto 1,Segmento,CUPO,CLASE DE RIESGO,PLAZO DIAS,Longitud,Latitud,Jefatura,Gerencia,Locacion,tipo_usuario,cupo_mean,cupo_max
20,1200288807,TIENDA LUCY,VACA RAMIREZ JAIME ALFREDO,2021,4,2021-04-03,107353,Plata +,0,CONTADO,0,-74.1464,4.5966,OS3,COS1,Bogota Sur,natural,1289052.5,1777476.0
21,1200288807,TIENDA LUCY,VACA RAMIREZ JAIME ALFREDO,2021,4,2021-04-07,100083,Plata +,0,CONTADO,0,-74.1464,4.5966,OS3,COS1,Bogota Sur,natural,1289052.5,1777476.0
22,1200288807,TIENDA LUCY,VACA RAMIREZ JAIME ALFREDO,2021,4,2021-04-10,116672,Plata +,0,CONTADO,0,-74.1464,4.5966,OS3,COS1,Bogota Sur,natural,1289052.5,1777476.0
23,1200288807,TIENDA LUCY,VACA RAMIREZ JAIME ALFREDO,2021,4,2021-04-14,87479,Plata +,0,CONTADO,0,-74.1464,4.5966,OS3,COS1,Bogota Sur,natural,1289052.5,1777476.0
24,1200288807,TIENDA LUCY,VACA RAMIREZ JAIME ALFREDO,2021,4,2021-04-17,100820,Plata +,0,CONTADO,0,-74.1464,4.5966,OS3,COS1,Bogota Sur,natural,1289052.5,1777476.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323981,1224316676,FRANCHOS PIZZA LOS ANGELES,ARCHILA GONZALEZ FRANCISCO JAVIER,2021,12,2021-12-15,93445,Plata +,0,CONTADO,0,-74.1192,4.6987,OB4,COB1,Bogota Norte,natural,685885.5,1617882.0
323982,1224316676,FRANCHOS PIZZA LOS ANGELES,ARCHILA GONZALEZ FRANCISCO JAVIER,2021,12,2021-12-18,219918,Plata +,0,CONTADO,0,-74.1192,4.6987,OB4,COB1,Bogota Norte,natural,685885.5,1617882.0
323983,1224316676,FRANCHOS PIZZA LOS ANGELES,ARCHILA GONZALEZ FRANCISCO JAVIER,2021,12,2021-12-22,152386,Plata +,0,CONTADO,0,-74.1192,4.6987,OB4,COB1,Bogota Norte,natural,685885.5,1617882.0
323984,1224316676,FRANCHOS PIZZA LOS ANGELES,ARCHILA GONZALEZ FRANCISCO JAVIER,2021,12,2021-12-24,51907,Plata +,0,CONTADO,0,-74.1192,4.6987,OB4,COB1,Bogota Norte,natural,685885.5,1617882.0


In [29]:
print("cantidad de casos en las que el cupo no seria suficiente con la media:",sum(ventas_cupo["Ingreso Bruto 1"]>ventas_cupo["cupo_mean"]))
print("cantidad de casos en las que el cupo no seria suficiente con el maximo:",sum(ventas_cupo["Ingreso Bruto 1"]>ventas_cupo["cupo_max"]))

cantidad de casos en las que el cupo no seria suficiente con la media: 780
cantidad de casos en las que el cupo no seria suficiente con el maximo: 259


In [30]:
print("Monto que no se podria cumplir con el cupo de la media:",sum((ventas_cupo["Ingreso Bruto 1"]-ventas_cupo["cupo_mean"])[ventas_cupo["Ingreso Bruto 1"]>ventas_cupo["cupo_mean"]]))
print("Monto que no se podria cubrir con el cupo del maximo:",sum((ventas_cupo["Ingreso Bruto 1"]-ventas_cupo["cupo_max"])[ventas_cupo["Ingreso Bruto 1"]>ventas_cupo["cupo_max"]]))


Monto que no se podria cumplir con el cupo de la media: 168565888.5
Monto que no se podria cubrir con el cupo del maximo: 46661767.5


In [31]:
len(ventas_cupo.Cliente[ventas_cupo["Ingreso Bruto 1"]>ventas_cupo["cupo_mean"]].unique())/len(ventas_cupo.Cliente.unique())

0.06689910655916322

In [32]:
### COn la media del ingreso y los hints
ventas_cupo.groupby(by=["Segmento"]).apply(lambda x: sum(x["Ingreso Bruto 1"]>x["cupo_mean"])/len(x))

Segmento
Plata       0.001728
Plata +     0.004239
dtype: float64

In [33]:
## Con la media del ingreso y el saldo
ventas_cupo.groupby(by=["Segmento"]).apply(lambda x: sum((x["Ingreso Bruto 1"]-x["cupo_mean"])[x["Ingreso Bruto 1"]>x["cupo_mean"]])/sum(x["Ingreso Bruto 1"]))

Segmento
Plata       0.004593
Plata +     0.010056
dtype: float64

In [34]:
### COn el maximo del ingreso y los hints
ventas_cupo.groupby(by=["Segmento"]).apply(lambda x: sum(x["Ingreso Bruto 1"]>x["cupo_max"])/len(x))

Segmento
Plata       0.001371
Plata +     0.000804
dtype: float64

In [35]:
## Con el maximo del ingreso y el saldo
ventas_cupo.groupby(by=["Segmento"]).apply(lambda x: sum((x["Ingreso Bruto 1"]-x["cupo_max"])[x["Ingreso Bruto 1"]>x["cupo_max"]])/sum(x["Ingreso Bruto 1"]))

Segmento
Plata       0.002306
Plata +     0.002209
dtype: float64

In [36]:
pd.read_parquet('s3://cnk-datalake/transacciones/Output/etl_json_unnest/')

,associated_data,transfer_id,associated_data_nested.location_id,associated_data_nested.piggybank_id,associated_data_nested.associated_costs,associated_data_nested.vault_id,associated_data_nested.bank_id,associated_data_nested.subject,associated_data_nested.ach_account_id,associated_data_nested.card_id,...,associated_data_nested.account_number,associated_data_nested.error,associated_data_nested.account_alias,associated_data_nested.cus,associated_data_nested.bulk_transaction_id,associated_data_nested.enterprise_id,associated_data_nested.sub,associated_data_nested.metadata,associated_data_nested.message,associated_data_nested.merchant_imei
0,"{""location_id"": ""40c9b7c4-7184-48e7-984f-f4526...",9acfda6b-59df-4de2-a985-2d3d13f2b8c1,40c9b7c4-7184-48e7-984f-f45260ef27a8,d4292324-9ceb-466d-ab5c-b1079e26834f,"{""gmf"":0.0,""iva"":0.0,""amount"":50.0,""iva_tx"":0....",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,"{""location_id"": ""40c9b7c4-7184-48e7-984f-f4526...",5cd8e62f-9084-401b-b1cb-f6fae984c9c8,40c9b7c4-7184-48e7-984f-f45260ef27a8,d4292324-9ceb-466d-ab5c-b1079e26834f,"{""gmf"":0.0,""iva"":0.0,""amount"":200.0,""iva_tx"":0...",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,"{""vault_id"": ""9d242634-cce5-4ad1-b5d4-fec6bdd1...",92c762dd-6c1a-47cd-b65d-82fc7323655d,a7f53aee-6fd6-45b3-8999-722109d9310e,f071ac69-aa36-407d-b1c7-5b210591f096,"{""gmf"":0.0,""iva"":0.0,""amount"":3900.0,""gmf_tx"":...",9d242634-cce5-4ad1-b5d4-fec6bdd1497a,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,"{""location_id"": ""899209e6-a251-4591-bfb5-5b1b1...",abf59059-b92b-4160-9a3e-981c27539f78,899209e6-a251-4591-bfb5-5b1b17d4cad6,388fb0e5-4941-4f58-baa0-796aeb859437,"{""gmf"":0.0,""iva"":0.0,""amount"":1000.0,""iva_tx"":...",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,"{""location_id"": ""40c9b7c4-7184-48e7-984f-f4526...",f2e349b0-5ad5-4d8d-91c9-9e3ec731ed85,40c9b7c4-7184-48e7-984f-f45260ef27a8,d4292324-9ceb-466d-ab5c-b1079e26834f,"{""gmf"":0.0,""iva"":0.0,""amount"":50.0,""gmf_tx"":0....",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215045,"{""location_id"": ""408a2716-eb72-4bb3-b131-7c4f3...",53b2d6ab-612b-4bda-8ae6-26d424eb25a8,408a2716-eb72-4bb3-b131-7c4f3d5eaac0,e4e2bf6c-37e3-4d0b-98c1-d786188c1694,"{""gmf"":0.0,""iva"":0.0,""amount"":1500.0,""iva_tx"":...",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
215046,"{""location_id"": ""0fbe6381-2052-4254-8e2b-3f920...",eb0e4ac4-84b7-4a01-83d5-034479a43199,0fbe6381-2052-4254-8e2b-3f9201769253,3c7a163b-f041-4abf-8355-0a758715f437,"{""gmf"":0.0,""iva"":0.0,""amount"":6200.0,""iva_tx"":...",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
215047,"{""location_id"": ""1e4f6734-a627-441b-b987-264c6...",09477393-fae1-4426-824a-f5440682bfca,1e4f6734-a627-441b-b987-264c607ca2be,4f1107f2-b5a7-4dea-be81-b2d94f61f765,"{""gmf"":0.0,""iva"":0.0,""amount"":200.0,""iva_tx"":0...",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
215048,"{""location_id"": ""553b2a76-e80a-4c88-84cd-cc14f...",1e740db3-56b4-47f7-adf0-06939bdd90f5,553b2a76-e80a-4c88-84cd-cc14f54c147e,68490096-13bf-43c9-ab1f-b893437bc91a,"{""gmf"":0.0,""iva"":0.0,""amount"":77300.0,""gmf_tx""...",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
